In [1]:
import ipywidgets as widgets
from IPython.display import display
import sys
import os

# Añadir la carpeta raíz del proyecto al sys.path
PROJECT_ROOT = os.path.abspath(os.path.join(os.getcwd(), ".."))
sys.path.append(PROJECT_ROOT)

# Importar funciones desde los scripts de la carpeta src
from src.local_indexing_faiss import load_and_process_pdfs, save_to_faiss
from src.local_retrieval_and_generation_faiss import query_rag
from src.get_embedding_function import get_embedding_function

print("✅ Entorno configurado correctamente.")

✅ Entorno configurado correctamente.


In [2]:
# Definir opciones para el usuario

# Embeddings
EMB_PROVIDERS = ["huggingface"]
MODELS_HUGGINGFACE = [
    "distiluse-base-multilingual-cased-v1", #135Mp
    "mrm8488/multilingual-e5-large-ft-sts-spanish-matryoshka-768-16-5e" #560Mp
]
MODELS_OPENAI = ["text-embedding-ada-002"]

# LLMs generadores de texto
LLM_PROVIDERS = ["groq", "huggingface"]
LLM_MODELS_GROQ = ["mixtral-8x7b-32768", "deepseek-r1-distill-llama-70b", "llama-3.2-1b-preview"]
LLM_MODELS_OPENAI = []#"gpt-3.5-turbo", "gpt-4"
LLM_MODELS_HUGGINGFACE = [
    "LenguajeNaturalAI/leniachat-gemma-2b-v0",
    "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B"# "deepseek-ai/DeepSeek-R1-Distill-Qwen-7B"
]

# Calcular la ruta absoluta del directorio 'data/pdfs'
PDF_FOLDER = os.path.join(PROJECT_ROOT, "data", "pdfs")
FAISS_INDEX_DIR = os.path.join(PROJECT_ROOT, "data", "faiss_index")

print(f"📂 Directorio de PDFs: {PDF_FOLDER}")
print(f"💾 Directorio de FAISS: {FAISS_INDEX_DIR}")

📂 Directorio de PDFs: c:\Users\zepol\Documents\UAM\TFM\extranjerIA\data\pdfs
💾 Directorio de FAISS: c:\Users\zepol\Documents\UAM\TFM\extranjerIA\data\faiss_index


# Indexing...

In [3]:
def update_model_options(change):
    """
    Actualiza las opciones de modelo en función del proveedor seleccionado.

    Args:
        change (dict): Información sobre el cambio en el selector de proveedor.
    """
    if change["new"] == "huggingface":
        model_selector.options = MODELS_HUGGINGFACE
    elif change["new"] == "openai":
        model_selector.options = MODELS_OPENAI

def update_llm_model_options(change):
    """
    Actualiza las opciones de modelos LLM según el proveedor seleccionado.

    Args:
        change (dict): Contiene la información sobre el cambio en el selector de proveedor.
    """
    if change["new"] == "groq":
        llm_model_selector.options = LLM_MODELS_GROQ
    elif change["new"] == "openai":
        llm_model_selector.options = LLM_MODELS_OPENAI
    elif change["new"] == "huggingface":
        llm_model_selector.options = LLM_MODELS_HUGGINGFACE

def run_indexing(model_name, provider):
    """
    Ejecuta el proceso de indexación de documentos en FAISS.

    Args:
        model_name (str): Modelo de embeddings a utilizar.
        provider (str): Proveedor del modelo ('huggingface' o 'openai').
    """
    
    try:
        print(f"📂 Procesando documentos desde: {PDF_FOLDER}")
        chunks = load_and_process_pdfs(PDF_FOLDER)
        if not chunks:
            raise ValueError("No se encontraron documentos para indexar.")
        print(f"📄 Se han generado {len(chunks)} fragmentos.")

        print("🔄 Guardando datos en FAISS...")
        save_to_faiss(chunks, model_name, provider)
        print("✅ Base de datos FAISS actualizada exitosamente.")
    except FileNotFoundError:
        print("❌ Error: La carpeta de PDFs no existe. Verifica la ruta.")
    except ValueError as ve:
        print(f"❌ Error: {ve}")
    except Exception as e:
        print(f"❌ Error inesperado durante la indexación: {e}")

def on_run_button_clicked(b):
    """
    Llama a la función de indexación con los valores seleccionados.

    Args:
        b (widgets.Button): Botón que activa la función.
    """
    run_indexing(model_selector.value, provider_selector.value)

In [4]:
# Widgets de selección para proveedor y modelo de embeddings
provider_selector = widgets.Dropdown(
    options=EMB_PROVIDERS,
    description="Proveedor:",
    disabled=False,
)

model_selector = widgets.Dropdown(
    description="Modelo:",
    disabled=False,
)

# Inicializar con modelos de Hugging Face por defecto
update_model_options({"new": provider_selector.value})

# Vincular la actualización al cambio de selección
provider_selector.observe(update_model_options, names="value")

# Botón para ejecutar la indexación
run_button = widgets.Button(
    description="Ejecutar Indexación",
    button_style="success"
)

# Widget de salida para mostrar resultados
output = widgets.Output()

# Asociar el botón con la función de ejecución
run_button.on_click(on_run_button_clicked)

📂 Procesando documentos desde: c:\Users\zepol\Documents\UAM\TFM\extranjerIA\data\pdfs
📄 Procesando: 16Familiar_comunitario_inicial.pdf.pdf
📄 Procesando: 17Familiar_comunitario_permanente.pdf.pdf
📄 Procesando: 1Arraigo_familiar_inicial.pdf.pdf
📄 Procesando: 1_Cuenta_ajena-inicial.pdf.pdf
📄 Procesando: 22bis_Estudiantes-ERASMUS.pdf.pdf
📄 Procesando: 26No_lucrativa_renovacion.pdf.pdf
📄 Procesando: 27Razones_humanitarias.pdf.pdf
📄 Procesando: 28Proteccion_internacional.pdf1.pdf
📄 Procesando: 29bisViolencia_genero_Tarjeta_provisional.pdf.pdf
📄 Procesando: 29Violencia_genero.pdf.pdf
📄 Procesando: 2Arraigo_social.pdf.pdf
📄 Procesando: 2_cuenta_ajena-renovacion.pdf.pdf
📄 Procesando: 3-_CUENTA_AJENA-modificacion.pdf.pdf
📄 Procesando: 4_Busqueda_empleo_proyecto_empresarial.pdf.pdf
📄 Procesando: 4_Cuenta_ajena-modificacion_otra_situacion.pdf.pdf
📄 Procesando: 5_Cuenta_propia-renovacion.pdf.pdf
📄 Procesando: 6_cuenta_propia-modificacion_otrascircunstancias.pdf.pdf
📄 Procesando: 7_cuenta_propia-mod

In [5]:
# Mostrar la interfaz en el notebook
display(provider_selector, model_selector, run_button, output)

Dropdown(description='Proveedor:', options=('huggingface',), value='huggingface')

Dropdown(description='Modelo:', options=('distiluse-base-multilingual-cased-v1', 'mrm8488/multilingual-e5-larg…

Button(button_style='success', description='Ejecutar Indexación', style=ButtonStyle())

Output()

# Retrieval and generation...

In [6]:
def display_query_interface():
    """
    Muestra la interfaz de usuario para hacer consultas al sistema RAG.
    Permite seleccionar proveedor de embeddings, modelo, LLM, y realizar consultas.
    """
    # Establecer un valor predeterminado para la consulta
    default_query = "¿Cuáles son los requisitos para la residencia en España?"

    # Widgets para seleccionar el proveedor y modelo de embeddings
    embedding_provider_selector = widgets.Dropdown(
        options=EMB_PROVIDERS,
        description="Emb. Prov:",
        disabled=False,
    )

    embedding_model_selector = widgets.Dropdown(
        options=MODELS_HUGGINGFACE,
        description="Emb. Mod:",
        disabled=False,
    )

    # Widgets para seleccionar el proveedor y modelo de LLM
    llm_provider_selector = widgets.Dropdown(
        options=LLM_PROVIDERS,
        description="LLM Prov:",
        disabled=False,
    )

    llm_model_selector = widgets.Dropdown(
        description="LLM Mod:",
        disabled=False,
    )

    # Función para actualizar modelos de embeddings en función del proveedor seleccionado
    def update_embedding_model_options(change):
        if change["new"] == "huggingface":
            embedding_model_selector.options = MODELS_HUGGINGFACE
        elif change["new"] == "openai":
            embedding_model_selector.options = MODELS_OPENAI

    # Función para actualizar modelos de LLM según el proveedor seleccionado
    def update_llm_model_options(change):
        if change["new"] == "groq":
            llm_model_selector.options = LLM_MODELS_GROQ
        elif change["new"] == "openai":
            llm_model_selector.options = LLM_MODELS_OPENAI
        elif change["new"] == "huggingface":
            llm_model_selector.options = LLM_MODELS_HUGGINGFACE

    # Asociar eventos de cambio a los selectores
    embedding_provider_selector.observe(update_embedding_model_options, names="value")
    llm_provider_selector.observe(update_llm_model_options, names="value")

    # Inicializar opciones de modelos con la selección actual
    update_embedding_model_options({"new": embedding_provider_selector.value})
    update_llm_model_options({"new": llm_provider_selector.value})

    # Widget para ingresar la consulta
    query_input = widgets.Textarea(
        value=default_query,
        description="Consulta:",
        layout=widgets.Layout(width='100%', height='80px')
    )

    # Botón para ejecutar la consulta
    run_button = widgets.Button(
        description="Ejecutar Consulta",
        button_style="success"
    )

    # Widget de salida para mostrar resultados
    output = widgets.Output()

    # Función para ejecutar la consulta
    def run_retrieval(b):
        # with output:
        #     output.clear_output()
        print(f"🔍 Ejecutando consulta con:")
        print(f"  - Embedding Provider: {embedding_provider_selector.value}")
        print(f"  - Embedding Model: {embedding_model_selector.value}")
        print(f"  - LLM Provider: {llm_provider_selector.value}")
        print(f"  - LLM Model: {llm_model_selector.value}")
        print(f"  - Pregunta: {query_input.value}")

        try:
            result = query_rag(
                query_input.value, 
                embedding_provider_selector.value, 
                embedding_model_selector.value, 
                llm_provider_selector.value, 
                llm_model_selector.value                    
            )

            print("\n📝 Respuesta generada:")
            print(result["response"])

            print("\n📄 Documentos fuente con puntuación:")
            for item in result["sources"]:
                print(f"Documento: {item['metadata']} - Score: {item['score']:.4f}")

            print("\n✅ Proceso de generación completado exitosamente.")
        except Exception as e:
            print(f"❌ Error durante la consulta: {e}")

    # Asociar el botón a la función de consulta
    run_button.on_click(run_retrieval)

    # Mostrar todos los widgets en la celda de ejecución
    display(embedding_provider_selector, embedding_model_selector, 
            llm_provider_selector, llm_model_selector, 
            query_input, run_button, output)

In [7]:
display_query_interface()

Dropdown(description='Emb. Prov:', options=('huggingface',), value='huggingface')

Dropdown(description='Emb. Mod:', options=('distiluse-base-multilingual-cased-v1', 'mrm8488/multilingual-e5-la…

Dropdown(description='LLM Prov:', options=('groq', 'huggingface'), value='groq')

Dropdown(description='LLM Mod:', options=('mixtral-8x7b-32768', 'deepseek-r1-distill-llama-70b', 'llama-3.2-1b…

Textarea(value='¿Cuáles son los requisitos para la residencia en España?', description='Consulta:', layout=Lay…

Button(button_style='success', description='Ejecutar Consulta', style=ButtonStyle())

Output()

🔍 Ejecutando consulta con:
  - Embedding Provider: huggingface
  - Embedding Model: distiluse-base-multilingual-cased-v1
  - LLM Provider: groq
  - LLM Model: llama-3.2-1b-preview
  - Pregunta: ¿Cuáles son los requisitos para la residencia en España?
⚙️ Cargando modelo de embeddings de Hugging Face: distiluse-base-multilingual-cased-v1
📂 Cargando índice FAISS desde: c:\Users\zepol\Documents\UAM\TFM\extranjerIA\data\faiss_index_huggingface_distiluse-base-multilingual-cased-v1


Eres un asistente experto en extranjería en España. Debes responder en **español** y basarte únicamente en el contexto siguiente:

***Contexto:

--Texto 1: Migraciones (inclusion.gob.es) 
• Padrón histórico, etc. (permanencia continuada en España durante los 2 años anteriores a la solicitud). 
• Compromiso de formación EX10. 
LABORAL. Más información en: Autorización de residencia temporal por circunstancias excepcionales. 
Arraigo laboral (HI 35) - Migraciones - Ministerio de Inclusión, Seguridad Social y Migrac

In [8]:
display_query_interface()

Dropdown(description='Emb. Prov:', options=('huggingface',), value='huggingface')

Dropdown(description='Emb. Mod:', options=('distiluse-base-multilingual-cased-v1', 'mrm8488/multilingual-e5-la…

Dropdown(description='LLM Prov:', options=('groq', 'huggingface'), value='groq')

Dropdown(description='LLM Mod:', options=('mixtral-8x7b-32768', 'deepseek-r1-distill-llama-70b', 'llama-3.2-1b…

Textarea(value='¿Cuáles son los requisitos para la residencia en España?', description='Consulta:', layout=Lay…

Button(button_style='success', description='Ejecutar Consulta', style=ButtonStyle())

Output()

🔍 Ejecutando consulta con:
  - Embedding Provider: huggingface
  - Embedding Model: distiluse-base-multilingual-cased-v1
  - LLM Provider: huggingface
  - LLM Model: deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B
  - Pregunta: ¿Cuáles son los requisitos para la residencia en España?
⚙️ Cargando modelo de embeddings de Hugging Face: distiluse-base-multilingual-cased-v1
📂 Cargando índice FAISS desde: c:\Users\zepol\Documents\UAM\TFM\extranjerIA\data\faiss_index_huggingface_distiluse-base-multilingual-cased-v1


Eres un asistente experto en extranjería en España. Debes responder en **español** y basarte únicamente en el contexto siguiente:

***Contexto:

--Texto 1: Migraciones (inclusion.gob.es) 
• Padrón histórico, etc. (permanencia continuada en España durante los 2 años anteriores a la solicitud). 
• Compromiso de formación EX10. 
LABORAL. Más información en: Autorización de residencia temporal por circunstancias excepcionales. 
Arraigo laboral (HI 35) - Migraciones - Ministerio de Inclusió

Device set to use cpu



📝 Respuesta generada:

Eres un asistente experto en extranjería en España. Debes responder en **español** y basarte únicamente en el contexto siguiente:

***Contexto:

--Texto 1: Migraciones (inclusion.gob.es) 
• Padrón histórico, etc. (permanencia continuada en España durante los 2 años anteriores a la solicitud). 
• Compromiso de formación EX10. 
LABORAL. Más información en: Autorización de residencia temporal por circunstancias excepcionales. 
Arraigo laboral (HI 35) - Migraciones - Ministerio de Inclusión, Seguridad Social y Migraciones 
(inclusion.gob.es)

----

--Texto 2: españoles…. 
 
3. Los titulares de los derechos a que se refieren los apartados anteriores que 
pretendan permanecer o fijar su residencia en España durante más de tres meses 
estarán obligados a solicitar un certificado de registro o una tarjeta de residencia de 
familiar de ciudadano de la Unión, según el procedimiento establecido en la 
presente norma. 
 
En su artículo 8, puntos 1 y 2, se establece que los 

In [ ]:
display_query_interface()

In [ ]:
display_query_interface()

In [ ]:
display_query_interface()

In [ ]:
display_query_interface()

In [ ]:
display_query_interface()

In [ ]:
display_query_interface()

In [ ]:
display_query_interface()